In [33]:
import open3d as o3d
import numpy as np
import cv2
from matplotlib import pyplot as plt

path = "IMG_DIR"
imgdir = f"{path}PHOTO"

In [34]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from PIL import Image
import torch
from transformers import GLPNFeatureExtractor, GLPNForDepthEstimation

In [26]:
feature_extractor = GLPNFeatureExtractor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

In [35]:
def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_src


In [36]:
def equalize_this(image_file, with_plot=False, gray_scale=False):
    
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image, g_image, b_image = cv2.split(image_src)

        r_image_eq = cv2.equalizeHist(r_image)
        g_image_eq = cv2.equalizeHist(g_image)
        b_image_eq = cv2.equalizeHist(b_image)

        image_eq = cv2.merge((r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
    else:
        image_eq = cv2.equalizeHist(image_src)
        cmap_val = 'gray'

    if with_plot:

        # Show the image with the depth color-coded
        cv2.imshow("Depth Colored", image_eq)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return image_eq


In [37]:
# image = equalize_this(image_file=imgdir, with_plot=False, gray_scale=True)
image = equalize_this(image_file=imgdir, with_plot=False, gray_scale=True)
org_img = read_this(image_file=imgdir, gray_scale=False)


In [38]:
img_size = image.shape
new_img = np.zeros((img_size[0], img_size[1], 3))

for i in range (img_size[0]):
    for j in range (img_size[1]):
        for k in range(3):
            new_img[i,j,k] = image[i,j]

In [39]:
# load and resize the input image
image = Image.fromarray(np.uint8(new_img)).convert('RGB')
image = Image.fromarray(new_img.astype('uint8'), 'RGB')

new_height = 480 if image.height > 480 else image.height
new_height -= (new_height % 32)
new_width = int(new_height * image.width / image.height)
diff = new_width % 32
new_width = new_width - diff if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

In [40]:
# prepare image for the model
inputs = feature_extractor(images=image, return_tensors="pt")

# get the prediction from the model
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

In [41]:
# remove borders
pad = 16
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))

In [43]:
# visualize the prediction
fig, ax = plt.subplots(1, 3)
ax[0].imshow(org_img)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(image)
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[2].imshow(output, cmap='plasma')
ax[2].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.pause(5)